# 24FS\_I4DS27: Adversarial Attacks \\ Wie kann KI überlistet werden? <br> 02-Models

In [1]:
%load_ext autoreload
%autoreload 2

import os
import torchvision
import torch
import torchmetrics

os.chdir("..")
from src.data.covidx import COVIDXDataModule
from src.models.imageclassifier import ImageClassifier

In [2]:
model = ImageClassifier(modelname="vit_l_32", output_size=1, p_dropout_classifier=0.2)

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((448, 448), antialias=True),
    ]
)

covidx_datamodule = COVIDXDataModule(path="data/raw/COVIDX-CXR4", transform=transform, batch_size=8, train_shuffle=True).setup()

---

In [3]:
for batch in covidx_datamodule.train_dataloader():
    image, label = batch
    prediction = model(image).squeeze(1)
    print(model.metrics(prediction, label))
    break

{'BinaryAccuracy': tensor(0.7500), 'BinaryPrecision': tensor(0.7500), 'BinaryRecall': tensor(1.), 'BinaryF1Score': tensor(0.8571), 'BinaryROC': (tensor([0.0000, 0.5000, 0.5000, 0.5000, 0.5000, 1.0000, 1.0000, 1.0000, 1.0000]), tensor([0.0000, 0.0000, 0.1667, 0.3333, 0.5000, 0.5000, 0.6667, 0.8333, 1.0000]), tensor([1.0000, 0.7178, 0.7119, 0.6838, 0.6525, 0.6162, 0.5939, 0.5729, 0.5629])), 'BinaryAUROC': tensor(0.2500), 'BinaryConfusionMatrix': tensor([[0, 2],
        [0, 6]]), 'BinarySpecificity': tensor(0.)}


In [4]:
class FoolingRatio(torchmetrics.Metric):
    # useful links: https://lightning.ai/docs/torchmetrics/stable/pages/implement.html
    pass

---